# AI Assistant Demo: Simple Math Tutor

Slightly evolved version of the math tutor example of OpenAI, see
https://platform.openai.com/docs/assistants/overview

Note that the assistant API is still in beta. Make sure that your kernel includes the most recent version of OpenAI: ``pip install --upgrade openai``

### Setup
Load the API keys and relevant Python libaries. Create an OpenAI client.

In [1]:
import os, time
import requests, json, dotenv
from openai import OpenAI

# Import API key from .env file
openai_api_key = (dotenv.dotenv_values()).get("OPENAI_API_KEY")
my_assistant_id = (dotenv.dotenv_values()).get("MATH_ASSISTANT_ID")
my_assistant_name = (dotenv.dotenv_values()).get("MATH_ASSISTANT_NAME")

client = OpenAI(api_key=openai_api_key)

### Look for existing assistant by id and name. If it does not exist, create one.

In [2]:
instructions_content = """You are a personal math tutor. 
When asked a question, write and run Python code to answer the question."""

assistant_ids = {}
for i, assistant in enumerate(client.beta.assistants.list()):
    assistant_ids[i] = assistant.id
    print(f"Assistant object {i}: '{assistant.id}' {assistant.name}")
    if assistant.id == my_assistant_id:
        print(f"Found assistant by id: {my_assistant_id}")
        break
    elif assistant.name == my_assistant_name:
        my_assistant_id = assistant.id
        print(f"Found assistant by name: {my_assistant_id}")
        break

print("\n------------------------------------------------------------ \n")
print(f"Assistant ID used: {my_assistant_id} \n")

if my_assistant_id is None:
    my_assistant = client.beta.assistants.create(
        model="gpt-4-1106-preview",
        instructions=instructions_content,
        name=my_assistant_name,
        tools=[{"type": "code_interpreter"}],
    )
    my_assistant_id = my_assistant.id
    print(f"New assistant created. Assistant object: {my_assistant} \n")



------------------------------------------------------------ 

Assistant ID used: None 

New assistant created. Assistant object: Assistant(id='asst_qIaXznHYdLbezzhfikwD4tji', created_at=1705589224, description=None, file_ids=[], instructions='You are a personal math tutor. \nWhen asked a question, write and run Python code to answer the question.', metadata={}, model='gpt-4-1106-preview', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')]) 



### Create a Thread

In [3]:
my_thread = client.beta.threads.create()
print(f"This is the thread object: {my_thread} \n")

This is the thread object: Thread(id='thread_Debxv7Pn31tHlwyze5Fzpht8', created_at=1705589235, metadata={}, object='thread') 



### Add a Message to a Thread

In [4]:
message_content = """I need to solve the equation `3x + 11 = 14`. Can you help me?"""

my_thread_message = client.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content=message_content,
)
print(f"This is the message object: {my_thread_message} \n")

This is the message object: ThreadMessage(id='msg_DT6K9yIGpdmxtPbN1azF72a0', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1705589237, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_Debxv7Pn31tHlwyze5Fzpht8') 



### Run the Assistant

In [5]:
my_run = client.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=my_assistant_id
)
print(f"This is the run object: {my_run} \n")

This is the run object: Run(id='run_WYLnfY2e41iBvZCo6C2qgy4y', assistant_id='asst_qIaXznHYdLbezzhfikwD4tji', cancelled_at=None, completed_at=None, created_at=1705589243, expires_at=1705589843, failed_at=None, file_ids=[], instructions='You are a personal math tutor. \nWhen asked a question, write and run Python code to answer the question.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_Debxv7Pn31tHlwyze5Fzpht8', tools=[ToolAssistantToolsCode(type='code_interpreter')]) 



### Periodically retrieve the Run to check on its status to see if it has moved to completed

In [6]:
status = my_run.status
while status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=my_thread.id,
        run_id=my_run.id
    )
    status=keep_retrieving_run.status
    time.sleep(0.2)
    print(".", end="")
print(f"\nRun status: {status}")

.......................
Run status: completed


### Retrieve the Messages added by the Assistant to the Thread

In [7]:
if status == "completed":
    all_messages = client.beta.threads.messages.list(
            thread_id=my_thread.id
    )
    print("\n------------------------------------------------------------ \n")
    print(f"User: {my_thread_message.content[0].text.value}")
    print(f"Assistant: {all_messages.data[0].content[0].text.value}")
else:
    print(f"Exiting with error. Status: {status}")


------------------------------------------------------------ 

User: I need to solve the equation `3x + 11 = 14`. Can you help me?
Assistant: The solution to the equation \(3x + 11 = 14\) is \(x = 1\).


### Detailed Run Steps of the Assistant

In [8]:
log = []
run_steps = client.beta.threads.runs.steps.list(
        thread_id=my_thread.id, run_id=my_run.id
    )
for i in run_steps.data:
    log.append(
              json.dumps(json.loads(i.step_details.model_dump_json()), indent=2)
    )
print("\n".join(log))

{
  "message_creation": {
    "message_id": "msg_IJ4xeQP1JaJEWLY0n7gXIOzF"
  },
  "type": "message_creation"
}
{
  "tool_calls": [
    {
      "id": "call_g575iEPx7ImqdhSSI905hpeZ",
      "code_interpreter": {
        "input": "from sympy import symbols, Eq, solve\n\n# Define the symbol\nx = symbols('x')\n\n# Define the equation\nequation = Eq(3*x + 11, 14)\n\n# Solve the equation\nsolution = solve(equation, x)\nsolution",
        "outputs": [
          {
            "logs": "[1]",
            "type": "logs"
          }
        ]
      },
      "type": "code_interpreter"
    }
  ],
  "type": "tool_calls"
}
